##### Copyright 2018 The TensorFlow Authors. [Licensed under the Apache License, Version 2.0](#scrollTo=y_UVSRtBBsJk).

In [ ]:
// #@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
// Licensed under the Apache License, Version 2.0 (the "License");
// you may not use this file except in compliance with the License.
// You may obtain a copy of the License at
//
// https://www.apache.org/licenses/LICENSE-2.0
//
// Unless required by applicable law or agreed to in writing, software
// distributed under the License is distributed on an "AS IS" BASIS,
// WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
// See the License for the specific language governing permissions and
// limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/swift/tutorials/model_training_walkthrough"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />TensorFlow.org에서 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/swift/blob/master/docs/site/tutorials/model_training_walkthrough.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/swift/blob/master/docs/site/tutorials/model_training_walkthrough.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />깃허브(GitHub)에서 소스 보기</a>
  </td>
</table>

# 모델 학습에 대한 연습

이 문서는 붓꽃의 품종을 분류하는 기계 학습 모델을 설계하여 텐서플로 스위프트를 설명하고 있습니다. 이는 텐서플로 스위프트를 다음과 같은 방식으로 사용합니다.
1. 모델을 구축하고,
2. 예시 데이터를 통해 모델을 학습시킨 후,
3. 학습되지 않는 미지의 데이터에 대한 예측을 생성하기 위해 이 전에 만든 모델을 사용합니다.

## 텐서플로 프로그래밍

이 문서는 다음과 같은 높은 수준의 텐서플로를 위한 스위프트 개념을 사용합니다.

* Datasets API를 사용해 데이터를 읽어들입니다.
* 스위프트 추상화를 활용해 모델을 설계합니다.
* 순수 스위프트 라이브러리가 여의치 않을 경우 스위프트-파이썬의 상호 운용성을 바탕으로 파이썬 라이브러리를 사용합니다. 

이 지침은 많은 TensorFlow 프로그램과 같이 구성되어 있습니다.

1. 데이터 집합을 불러들여 분석합니다.
2. 사용할 모델의 종류를 선정합니다.
3. 모델을 학습시킵니다.
4. 모델의 효율성을 평가하십시오.
5. 예측 생성을 위해 이전에 학습한 모델을 사용하십시오.

## 기본 설정 프로그램

### 기본 환경 및 기능 설정하기

텐서플로와 유용한 파이썬 모듈을 임포트하세요.

In [ ]:
import TensorFlow

import Python
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")
let plt = Python.import("matplotlib.pyplot")

In [ ]:
// 몇 가지 주요 제한 사항을 해결하는 데 도움이 되는 도움말 파일을 데이터세트 API에서 다운로드하세요
import Foundation
import FoundationNetworking
func download(from sourceString: String, to destinationString: String) {
    let source = URL(string: sourceString)!
    let destination = URL(fileURLWithPath: destinationString)
    let data = try! Data.init(contentsOf: source)
    try! data.write(to: destination)
}
download(
    from: "https://raw.githubusercontent.com/tensorflow/swift/master/docs/site/tutorials/TutorialDatasetCSVAPI.swift",
    to: "TutorialDatasetCSVAPI.swift")

## 붓꽃 분류 문제

당신이 찾은 각각의 붓꽃을 분류하기 위한 자동화 방법을 찾고 있는 식물학자라고 상상해봅시다. 기계학습은 꽃을 통계적으로 분류하는 많은 알고리즘을 제공합니다. 예를 들어, 정교한 기계학습 프로그램은 사진을 바탕으로 꽃을 분류할 수 있습니다. 아직 목표가 너무 겸손합니다. - 우리는 [꽃받침](https://en.wikipedia.org/wiki/Sepal)과 [꽃잎](https://en.wikipedia.org/wiki/Petal)의 길이, 넓이를 측정한 것을 바탕으로 붓꽃을 분류할 것입니다.

붓꽃 속에는 약 300여종이 포함되어 있지만, 이 프로그램은 다음 세 종만을 분류할 것입니다 : 

* Iris setosa
* Iris virginica
* Iris versicolor

<table>
  <tr><td>
    <img src="https://www.tensorflow.org/images/iris_three_species.jpg"
         alt="Petal geometry compared for three iris species: Iris setosa, Iris virginica, and Iris versicolor">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> <a href="https://commons.wikimedia.org/w/index.php?curid=170298">Iris setosa</a> (by <a href="https://commons.wikimedia.org/wiki/User:Radomil">Radomil</a>, CC BY-SA 3.0), <a href="https://commons.wikimedia.org/w/index.php?curid=248095">Iris versicolor</a>, (by <a href="https://commons.wikimedia.org/wiki/User:Dlanglois">Dlanglois</a>, CC BY-SA 3.0), and <a href="https://www.flickr.com/photos/33397993@N05/3352169862">Iris virginica</a> (by <a href="https://www.flickr.com/photos/33397993@N05">Frank Mayfield</a>, CC BY-SA 2.0).<br/>&nbsp;
  </td></tr>
</table>

다행히도, 누군가 이미 꽃받침과 꽃잎 수치를 활용한 [120개의 붓꽃 데이터셋](https://en.wikipedia.org/wiki/Iris_flower_data_set)을 만들었습니다. 이는 초보자를 위한 기계학습 분류 문제로 유명한 고전 데이터셋이라고 볼 수 있습니다.

## 훈련 데이터셋을 임포트하고 분석하기

데이터셋 파일을 다운로드한 후 스위프트 프로그램에서 사용할 수 있는 구조로 변환합니다.

### 데이터셋 다운로드

 http://download.tensorflow.org/data/iris_training.csv 에서 훈련 데이터셋 파일을 다운로드합니다.

In [ ]:
let trainDataFilename = "iris_training.csv"
download(from: "http://download.tensorflow.org/data/iris_training.csv", to: trainDataFilename)

### 데이터 조사하기

이 데이터셋 `iris_training.csv`는 쉼표로 구분된 값(CSV)으로 포맷된 표식 데이터를 저장하는 일반 텍스트 파일입니다. 처음 5개 항목을 살펴보세요.

In [ ]:
let f = Python.open(trainDataFilename)
for _ in 0..<5 {
    print(Python.next(f).strip())
}
f.close()

이 데이터셋의 보기에서 해당 사항을 주의하세요. : 

1.   첫 번째 줄은 데이터셋에 관한 정보를 포함하는 전처리문입니다.
  * 총 120개의 예시가 있는데 각 예시는 네 가지 특징과 세 가지 가능한 라벨 이름 중 하나를 가지고 있습니다.
2. 후속 행은 데이터 레코드로, 각 행에 [*예시*](https://developers.google.com/machine-learning/glossary/#example)가 하나씩 있으며, 여기서:
  * 첫 번째 네 개의 필드는 [*특징*](https://developers.google.com/machine-learning/glossary/#feature)입니다. 이것들은 예시의 특징인데 여기서, 필드는 꽃의 측정 결과값을 나타내는 실수값을 가지고 있습니다.
  * 마지막 열은 [*라벨*](https://developers.google.com/machine-learning/glossary/#label)입니다. 이는 우리가 예측하고 싶어하는 값인데 이 데이터셋에서는 꽃 이름에 해당하는 0, 1, 2의 정수값입니다.

이를 코드로 써봅시다 : 

In [ ]:
let featureNames = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
let labelName = "species"
let columnNames = featureNames + [labelName]

print("Features: \(featureNames)")
print("Label: \(labelName)")

각 라벨은 문자열 이름과 연관이 되어있지만(예를 들어, "setosa"), 기계학습은 전형적으로 숫자 형태의 값에 의존합니다. 라벨 숫자는 다음과 같이 명명된 이름에 매칭됩니다.

* `0`: Iris setosa
* `1`: Iris versicolor
* `2`: Iris virginica

특징과 라벨에 대한 더 많은 정보를 원한다면, 기계 학습 충돌 관련 강의의 [ML 용어](https://developers.google.com/machine-learning/crash-course/framing/ml-terminology) 부분을 참조하세요.


In [ ]:
let classNames = ["Iris setosa", "Iris versicolor", "Iris virginica"]

### 데이터 세트 생성

스위프트용 텐서플로 데이터 세트 API는 데이터를 읽고, 훈련할 때 사용되는 형태로 변환하기 위한 상위 수준의 API입니다. 

결국, 데이터 세트 API는 많은 파일 형식에서 데이터를 불러올 수 있게 될 것입니다. 데이터 세트 API는 현재 매우 불완전하기 때문에 CSV 파일의 데이터 세트 API의 데이터를 `IrisBatch`구조로 불러오는 데 도움이 되는 "TutorialDatasetCSVAPI.swift"의 글루 코드(호환성이 없는 부분끼리 결합하기 위해 작동하는 코드를 의미함)를 포함합니다.

In [ ]:
let batchSize = 32

/// 아이리스 데이터 세트의 예제 모음입니다.
struct IrisBatch {
    /// [batchSize, featureCount] 는 텐서의 특성입니다.
    let features: Tensor<Float>

    /// [batchSize] 는 텐서의 레이블입니다.
    let labels: Tensor<Int32>
}

%include "TutorialDatasetCSVAPI.swift"

let trainDataset: Dataset<IrisBatch> = Dataset(
    contentsOfCSVFile: trainDataFilename, hasHeader: true,
    featureColumns: [0, 1, 2, 3], labelColumns: [4]
).batched(batchSize)

`Dataset<IrisBatch>`는 `IrisBatch`를 차례로 배열한 것입니다. 데이터 세트의 첫 번째 요소를 살펴보겠습니다.


In [ ]:
let firstTrainExamples = trainDataset.first!
let firstTrainFeatures = firstTrainExamples.features
let firstTrainLabels = firstTrainExamples.labels
print("First batch of features: \(firstTrainFeatures)")
print("First batch of labels: \(firstTrainLabels)")

첫 번째 `batchsize` 예시의 특성은 `firstTrainFeatures`으로 분류되어 있고(또는 *배치되어 있고*), 첫 번째 `batchSize` 예시의 레이블은 `firstTrainLabels`과 배치되어 있다는 점에 유의해야 합니다.

파이썬의 matplotlib를 사용하여 배치에서 몇 가지 특성을 표시하면 일부 클러스터를 볼 수 있습니다.


In [ ]:
let firstTrainFeaturesTransposed = firstTrainFeatures.transposed()
let petalLengths = firstTrainFeaturesTransposed[2].scalars
let sepalLengths = firstTrainFeaturesTransposed[0].scalars

plt.scatter(petalLengths, sepalLengths, c: firstTrainLabels.array.scalars)
plt.xlabel("Petal length")
plt.ylabel("Sepal length")
plt.show()

## 모델 유형 선택

### 왜 모델인가?

[*모델*](https://developers.google.com/machine-learning/crash-course/glossary#model)은 특성과 레이블 사이의 관계입니다. 붓꽃 분류 문제의 경우, 모델은 꽃받침과 꽃잎 측정 사이의 관계를 정의하고 붓꽃의 종을 예측하게 됩니다. 일부 간단한 모델들은 대수학 몇 줄로 설명할 수 있지만, 복잡한 기계학습 모델들은 요약하기 어려운 매개 변수가 많습니다.

기계 학습을 *사용하지 않고* 4가지 특성과 붓꽃 종의 관계를 결정할 수 있습니까? 즉, 기존의 프로그래밍 기법(예: 많은 조건문)을 사용하여 모델을 만들 수 있습니까? 아마도, 특정 종에 대한 꽃받침과 꽃잎 측정 사이의 관계를 결정할 수 있을 만큼 충분히 긴 데이터 세트를 분석했다면 가능할 것입니다. 그리고 더 복잡한 데이터 세트에서는 불가능할 수도 있을 것입니다. 좋은 기계 학습 접근법이 사용자를 위한 모델을 결정합니다. 만약 사용자가 적절한 기계 학습 모델 타입으로 대표되는 예들을 제공한다면, 프로그램은 사용자를 위한 관계를 알아낼 것입니다.

### 모델 선택

우리는 훈련할 모델을 선택해야 합니다. 많은 유형의 모델 중에서 알맞은 모델을 고르는 것은 경험을 필요로 합니다. 이 튜토리얼에서는 붓꽃 분류 문제를 해결하기 위해 신경망을 이용합니다. [*신경망*](https://developers.google.com/machine-learning/glossary/#neural_network)은 특성과 레이블 사이의 복잡한 관계를 찾을 수 있습니다. 이 그래프는 고도로 구조화된 그래프로서, 하나 이상의 [*히든 레이어*](https://developers.google.com/machine-learning/glossary/#hidden_layer)로 구성되어 있습니다. 각각의 히든 레이어는 하나 이상의 [*뉴런*](https://developers.google.com/machine-learning/glossary/#neuron)으로 구성되어 있습니다. 신경망에는 몇 가지 범주가 있고, 이 프로그램은 밀도나 [*완전 연결 신경망*](https://developers.google.com/machine-learning/glossary/#fully_connected_layer)을 사용합니다. 한 층의 뉴런은 이전 층의 모든 뉴런으로부터 입력 연결을 받습니다. 예를 들어, 그림 2는 입력 레이어, 히든 레이어 및 출력 레이어로 구성된 밀집 신경망을 표현하였습니다.

<table>
  <tr><td>
    <img src="https://www.tensorflow.org/images/custom_estimators/full_network.png"
         alt="A diagram of the network architecture: Inputs, 2 hidden layers, and outputs">
  </td></tr>
  <tr><td align="center">
    <b>Figure 2.</b> A neural network with features, hidden layers, and predictions.<br/>&nbsp;
  </td></tr>
</table>

그림 2의 모델을 학습 시켜 라벨이 없는 예에 적용하면 세 가지 예측을 수행할 수 있습니다. 이 꽃이 주어진 붓꽃일 가능성을 예측하는 것을 [*추론*](https://developers.google.com/machine-learning/crash-course/glossary#inference)이라고 합니다. 출력 예측의 합이 1.0이라고 예를 들어보겠습니다. 그림 2에서, 이 예측은 *Iris setosa*인 경우 `0.02`, *Iris versicolor*인 경우 `0.95`, *Iris virginica*인 경우 `0.03`으로 나누어집니다. 이 모델이 95% 확률로 라벨이 없는 예의 꽃이 *Iris versicolor*라고 예측한다는 것을 의미합니다.

### 텐서플로 딥러닝 라이브러리용 스위프트를 이용하여 모델 생성

[스위프트를 위한 텐서플로 딥러닝 라이브러리](https://github.com/tensorflow/swift-apis)에서는 원시 층과 이들을 함께 연결하기 위한 규칙을 정의하고 있어서 모델 제작과 실험이 용이합니다. 

이 모델은 [`레이어`](https://www.tensorflow.org/swift/api_docs/Protocols/Layer)에 부합한 `구조`로서, 입력`텐서`에서 출력`텐서`로 매핑하는  [`callAsFunction(_:)`](https://www.tensorflow.org/swift/api_docs/Protocols/Layer#call_:) 방법을 정의한다는 것을 의미합니다. `callAsFunction(_:)` 방법은 흔히 밀집 레이어를 통해 입력의 순서를 단순화시킵니다. 세 개의 [`밀집`](https://www.tensorflow.org/swift/api_docs/Structs/Dense)된 레이어를 통해 입력을 배열하는 `아이리스 모델`을 정의해 보겠습니다.

In [ ]:
import TensorFlow

let hiddenSize: Int = 10
struct IrisModel: Layer {
    var layer1 = Dense<Float>(inputSize: 4, outputSize: hiddenSize, activation: relu)
    var layer2 = Dense<Float>(inputSize: hiddenSize, outputSize: hiddenSize, activation: relu)
    var layer3 = Dense<Float>(inputSize: hiddenSize, outputSize: 3)
    
    @differentiable
    func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
        return input.sequenced(through: layer1, layer2, layer3)
    }
}

var model = IrisModel()

활성화 함수는 층에 있는 각 노드의 출력 모양을 결정합니다. 비선형성이 없다면 모델은 단일 층과 동일해지므로 비선형성은 중요합니다. 이용 가능한 활성화는 많지만, [ReLU](https://www.tensorflow.org/swift/api_docs/Functions#/s:10TensorFlow4reluyAA0A0VyxGAESFRzAA0aB6ScalarRzlF)는 히든 레이어에 일반적입니다.

히든 레이어와 뉴런의 이상수는 문제와 데이터 세트에 따라 결정됩니다. 기계학습의 많은 측면과 마찬가지로, 신경망의 가장 좋은 형태를 고르는 것은 지식과 실험의 혼합을 필요로 합니다. 경험법칙과 같이 히든 레이어와 뉴런의 수를 늘리면 일반적으로 더 강력한 모델이 만들어지는데, 효과적인 훈련을 위해서는 더 많은 데이터를 필요로 합니다.

### 모델 사용

이 모델이 기능의 배치에 어떤 영향을 미치는 지 간단하게 살펴보도록 하겠습니다: 

In [ ]:
// 모델을 특성 배치에 적용하세요.
let firstTrainPredictions = model(firstTrainFeatures)
firstTrainPredictions[0..<5]

여기서, 각 예는 각 클래스에 대해 [로짓](https://developers.google.com/machine-learning/crash-course/glossary#logits)을 반환합니다.

이러한 로짓을 각 클래스에 대한 확률로 변환하려면 [소프트맥스](https://developers.google.com/machine-learning/crash-course/glossary#softmax) 함수를 사용해야합니다 :

In [ ]:
softmax(firstTrainPredictions[0..<5])

`argmax`를 클래스에 거쳐 적용하면 예측된 클래스 인덱스를 얻을 수 있습니다. 하지만 모델은 아직 훈련되지 않았기 때문에, 좋은 예측은 아닙니다.

In [ ]:
print("Prediction: \(firstTrainPredictions.argmax(squeezingAxis: 1))")
print("    Labels: \(firstTrainLabels)")

## 모델 훈련하기

[*훈련*](https://developers.google.com/machine-learning/crash-course/glossary#training)은 모델이 점차 최적화되거나 모델이 데이터셋을 *학습* 하는 기계학습의 단계입니다. 목표는 보이지 않은 데이터에 대한 예측을 위해 교육 데이터 셋의 구조에 대해 충분히 배우는 것입니다. 교육 데이터 셋에 대해 *너무 많이* 학습되면 예측은 지금까지 본 데이터에만 적용되므로 일반화할 수 없을 것입니다. 이 문제를 [*과적합*](https://developers.google.com/machine-learning/crash-course/glossary#overfitting)이라고 합니다. 이것은 문제를 해결하는 방법을 이해하는 대신 답을 암기하는 것과 같습니다.

붗꽃 분류 문제는 [*지도 학습*](https://developers.google.com/machine-learning/glossary/#supervised_machine_learning)의 한 예입니다: 모델은 라벨을 포함하는 예제로 훈련됩니다. [*비지도 학습*](https://developers.google.com/machine-learning/glossary/#unsupervised_machine_learning)에서 예제는 라벨을 포함하지 않습니다. 대신 모델은 전형적으로 특징들 사이의 패턴을 발견합니다.

### 손실 함수 선택

교육과 평가 단계 모두 모델의 [*손실*](https://developers.google.com/machine-learning/crash-course/glossary#loss)을 계산해야 합니다. 이것은 모델의 예측이 원하는 라벨로부터 얼마나 벗어난지를 측정합니다. 다시 말해서, 모델이 얼마나 나쁜지 측정합니다. 우리는 이 값이 최소화하거나 최적화하기를 원합니다.

우리의 모델은 모델의 클래스 확률 예측과 원하는 라벨을 가져와서 예제 전체에 걸쳐 평균 손실을 반환하는 [`softmaxCrossEntropy(logits:labels:)`](https://www.tensorflow.org/swift/api_docs/Functions#/s:10TensorFlow19softmaxCrossEntropy6logits6labelsAA0A0VyxGAG_AFys5Int32VGtAA0aB13FloatingPointRzlF)함수를 사용하여 손실을 계산할 것입니다.

현재 훈련되지 않은 모델의 손실을 계산합니다:

In [ ]:
let untrainedLogits = model(firstTrainFeatures)
let untrainedLoss = softmaxCrossEntropy(logits: untrainedLogits, labels: firstTrainLabels)
print("Loss test: \(untrainedLoss)")

### 최적화 만들기

[*옵티마이저*](https://developers.google.com/machine-learning/crash-course/glossary#optimizer)는 계산된 경사를 모델의 변수에 적용하여 `손실` 함수를 최소화합니다. 손실 함수를 곡선 표면(그림 3 참조)으로 생각할 수 있으며, 우리는 이동하면서 가장 낮은 지점을 찾고자 합니다. 경사도는 가장 가파른 오르막길을 가리킵니다. 그래서 우리는 반대 방향으로 여행하고 언덕 아래로 내려갈 것입니다. 각 배치의 손실과 구배를 반복적으로 계산함으로써, 우리는 훈련 중에 모델을 조정할 것입니다. 점차적으로, 모델은 손실을 최소화하기 위해 가중치와 편향이 가장 잘 조합된 것을 발견합니다. 그리고 손실이 낮을 수록 모델의 에측이 좋아집니다.

<table>
  <tr><td>
    <img src="https://cs231n.github.io/assets/nn3/opt1.gif" width="70%"
         alt="Optimization algorithms visualized over time in 3D space.">
  </td></tr>
  <tr><td align="center">
    <b>Figure 3.</b> Optimization algorithms visualized over time in 3D space.<br/>(Source: <a href="http://cs231n.github.io/neural-networks-3/">Stanford class CS231n</a>, MIT License, Image credit: <a href="https://twitter.com/alecrad">Alec Radford</a>)
  </td></tr>
</table>

텐서플로를 위한 스위프트에는 훈련에 사용할 수 있는 많은 [옵티마이저 알고리즘](https://github.com/rxwei/DeepLearning/blob/master/Sources/DeepLearning/Optimizer.swift)이 있습니다. 이 모델은 [*미니 배치 확률적 경사하강법*](https://developers.google.com/machine-learning/crash-course/glossary#gradient_descent) (SGD) 알고리즘을 구현하는 SGD 최적기를 사용합니다. `learningRate`는 언덕 아래 각 반복에 대해 취할 단계 크기를 설정합니다. 이것은 여러분이 일반적으로 더 나은 결과를 얻기 위해 조절할 수 있는 하이퍼 파라미터 입니다.

In [ ]:
let optimizer = SGD(for: model, learningRate: 0.01)

`옵티마이저`를 이용해 한 번의 경사 강하 단계를 수행합니다. 첫번째로, 모델에 대한 손실의 경사를 계산합니다:

In [ ]:
let (loss, grads) = model.valueWithGradient { model -> Tensor<Float> in
    let logits = model(firstTrainFeatures)
    return softmaxCrossEntropy(logits: logits, labels: firstTrainLabels)
}
print("Current loss: \(loss)")

다음으로, 우리는 방금 계산한 경사를 옵티마이저에 전달하고, 이에 따라 모델의 다른 변수들을 업데이트합니다 :

In [ ]:
optimizer.update(&model.allDifferentiableVariables, along: grads)

손실을 다시 계산한다면, 그것은 더 작아야 합니다. 왜냐하면 일반적으로 경사 하강 단계가 손실을 감소시키기 때문입니다.

In [ ]:
let logitsAfterOneStep = model(firstTrainFeatures)
let lossAfterOneStep = softmaxCrossEntropy(logits: logitsAfterOneStep, labels: firstTrainLabels)
print("Next loss: \(lossAfterOneStep)")

### 훈련 루프

모든 조각들을 제자리에 두고, 모델은 훈련을 받을 준비가 되었습니다! 훈련 루프는 더 나은 예측을 위해 데이터셋 예제를 모델에 공급합니다. 다음의 코드 블록은 훈련단계들을 설정합니다.

1. 각 세대를 반복합니다. 데이터셋을 한 번 통과하는 것이 한 세대입니다.
2. 한 세대에서 *특징*(`x`)과 *레이블*(`y`)을 잡는 `데이터셋` 훈련에 대한 각 예를 반복합니다.
3. 예시의 특성을 이용하여, 예측하고 레이블과 비교합니다. 예측의 부정확성을 측정하고 이를 사용하여 모델의 손실과 경사를 계산합니다.
4. 경사하강법을 이용하여 모델의 변수를 업데이트합니다.
5. 시각화를 위해 몇 가지 통계를 추적합니다.
6. 각 세대를 반복합니다.

`epochCount` 변수는 데이터 집합을 순환하는 횟수를 의미합니다. 반대로, 모델을 더 오래 훈련한다고 해서 더 나은 모델이 보장되는 것은 아닙니다. `epochCount`는 튜닝할 수 있는 [*초매개변수*](https://developers.google.com/machine-learning/glossary/#hyperparameter) 입니다. 정확한 숫자를 선택하려면 보통 경험과 실험이 필요합니다. 

In [ ]:
let epochCount = 500
var trainAccuracyResults: [Float] = []
var trainLossResults: [Float] = []

In [ ]:
func accuracy(predictions: Tensor<Int32>, truths: Tensor<Int32>) -> Float {
    return Tensor<Float>(predictions .== truths).mean().scalarized()
}

for epoch in 1...epochCount {
    var epochLoss: Float = 0
    var epochAccuracy: Float = 0
    var batchCount: Int = 0
    for batch in trainDataset {
        let (loss, grad) = model.valueWithGradient { (model: IrisModel) -> Tensor<Float> in
            let logits = model(batch.features)
            return softmaxCrossEntropy(logits: logits, labels: batch.labels)
        }
        optimizer.update(&model.allDifferentiableVariables, along: grad)
        
        let logits = model(batch.features)
        epochAccuracy += accuracy(predictions: logits.argmax(squeezingAxis: 1), truths: batch.labels)
        epochLoss += loss.scalarized()
        batchCount += 1
    }
    epochAccuracy /= Float(batchCount)
    epochLoss /= Float(batchCount)
    trainAccuracyResults.append(epochAccuracy)
    trainLossResults.append(epochLoss)
    if epoch % 50 == 0 {
        print("Epoch \(epoch): Loss: \(epochLoss), Accuracy: \(epochAccuracy)")
    }
}

### 시간에 따른 손실함수 시각화

모델의 교육 진행 상황을 출력하는 것은 도움이 되지만, 종종 이러한 진행 상황을 보는 것이 더 도움이 됩니다. 파이썬의 `matplotlib` 모듈을 이용해 기본 차트를 만들 수 있습니다.

이 차트를 해석하는 것은 약간의 경험이 필요하지만, 당신은 *손실*이 내려가고 *정확도*가 올라가는 것을 보기 원할 것입니다.

In [ ]:
plt.figure(figsize: [12, 8])

let accuracyAxes = plt.subplot(2, 1, 1)
accuracyAxes.set_ylabel("Accuracy")
accuracyAxes.plot(trainAccuracyResults)

let lossAxes = plt.subplot(2, 1, 2)
lossAxes.set_ylabel("Loss")
lossAxes.set_xlabel("Epoch")
lossAxes.plot(trainLossResults)

plt.show()

그래프의 y축은 0이 아니라는 점에 유의하세요.

## 모델의 효과 평가하기

이제 그 모델이 훈련되었으니, 우리는 그 모델의 성능에 대한 몇 가지 통계를 얻을 수 있습니다.

*평가*는 모형이 예측을 얼마나 효과적으로 하는지를 결정하는 것을 의미합니다. 아이리스 분류에서 모델의 효과를 결정하려면, 일부 세팔 및 꽃잎 측정치를 모델에 전달하고 모형에 그들이 나타내는 아이리스 종류를 예측하도록 요청하세요. 그 후 모델의 예측을 실제 라벨과 비교합니다. 예를 들어, 입력 예제의 절반에 맞는 종을 고른 모델은 `0.5`의 [*정확도*](https://developers.google.com/machine-learning/glossary/#accuracy)를 가지고 있습니다. 그림 4는 예측 5개 중 4개를 80%의 정확도로 정확하게 계산하여 약간 더 효과적인 모델을 보여주고 있습니다.

<table cellpadding="8" border="0">
  <colgroup>
    <col span="4" >
    <col span="1" bgcolor="lightblue">
    <col span="1" bgcolor="lightgreen">
  </colgroup>
  <tr bgcolor="lightgray">
    <th colspan="4">Example features</th>
    <th colspan="1">Label</th>
    <th colspan="1" >Model prediction</th>
  </tr>
  <tr>
    <td>5.9</td><td>3.0</td><td>4.3</td><td>1.5</td><td align="center">1</td><td align="center">1</td>
  </tr>
  <tr>
    <td>6.9</td><td>3.1</td><td>5.4</td><td>2.1</td><td align="center">2</td><td align="center">2</td>
  </tr>
  <tr>
    <td>5.1</td><td>3.3</td><td>1.7</td><td>0.5</td><td align="center">0</td><td align="center">0</td>
  </tr>
  <tr>
    <td>6.0</td> <td>3.4</td> <td>4.5</td> <td>1.6</td> <td align="center">1</td><td align="center" bgcolor="red">2</td>
  </tr>
  <tr>
    <td>5.5</td><td>2.5</td><td>4.0</td><td>1.3</td><td align="center">1</td><td align="center">1</td>
  </tr>
  <tr><td align="center" colspan="6">
    <b>Figure 4.</b> An iris classifier that is 80% accurate.<br/>&nbsp;
  </td></tr>
</table>

### 테스트 데이터셋 설정

모델을 평가하는 것은 모델을 훈련시키는 것과 비슷합니다. 가장 큰 차이점은 예시들이 학습 세트 보다는 별도의 [*테스트 셋*](https://developers.google.com/machine-learning/crash-course/glossary#test_set) 로부터 나온다는 것입니다. 모델의 효과성을 공정하게 평가하기 위해, 모델을 평가하는 데 사용되는 예는 모델을 훈련시키는 데 사용되는 예와 달라야 합니다. 모델의 효과성을 공정하게 평가하기 위해, 모델을 평가하는 데 사용되는 예는 모델을 훈련시키는 데 사용되는 예와 달라야 합니다.

테스트용 `데이터셋`의 설정은 훈련용`데이터셋`의 설정과 유사합니다. http://download.tensorflow.org/data/iris_test.csv 에서 테스트 셋을 다운로드 할 수 있습니다.

In [ ]:
let testDataFilename = "iris_test.csv"
download(from: "http://download.tensorflow.org/data/iris_test.csv", to: testDataFilename)

 이제 `데이터셋`에 업로드 합니다 :

In [ ]:
let testDataset: Dataset<IrisBatch> = Dataset(
    contentsOfCSVFile: testDataFilename, hasHeader: true,
    featureColumns: [0, 1, 2, 3], labelColumns: [4]
).batched(batchSize)

### 테스트 데이터세트에서 모델 평가하기

모델은 훈련 단계와 달리 시험 데이터의 [세대](https://developers.google.com/machine-learning/glossary/#epoch)만 평가합니다. 다음 코드 셀에서 테스트 세트의 각 예제에 대해 반복하고 모델의 예측을 실제 라벨과 비교합니다. 이것은 전체 시험 세트에 걸쳐 모델의 정확도를 측정하는 데 사용됩니다.

In [ ]:
// NOTE: 테스트 데이터 세트에 'batchSize = 32'와 30개의 예제가 있는 경우, 루프에서 한 개의 배치만 실행됩니다.
for testBatch in testDataset {
    let logits = model(testBatch.features)
    let predictions = logits.argmax(squeezingAxis: 1)
    print("Test batch accuracy: \(accuracy(predictions: predictions, truths: testBatch.labels))")
}

예를 들어, 우리는 대게 첫 번째 배치에서 모델이 정확하다는 것을 볼 수 있습니다 :

In [ ]:
let firstTestBatch = testDataset.first!
let firstTestBatchLogits = model(firstTestBatch.features)
let firstTestBatchPredictions = firstTestBatchLogits.argmax(squeezingAxis: 1)

print(firstTestBatchPredictions)
print(firstTestBatch.labels)

## 예측을 위해 학습된 모델 사용하기

우리는 아이리스 종류를 분류하는 것이 좋지만 완벽하지는 않다는 것을 모델을 훈련시키고 증명했습니다. 이제 훈련된 모델을 사용하여 레이블이 없는 예제, 즉 형상이 포함되어 있지만 [레이블이 없는 예제](https://developers.google.com/machine-learning/glossary/#unlabeled_example)에 대해 몇 가지 예측을 해 봅니다.

실생활에서, 라벨이 없는 예는 앱, CSV 파일, 데이터 피드를 포함한 많은 다른 소스에서 나올 수 있습니다. 현재로서는 라벨이 없는 세 가지 예를 수동으로 제공하여 라벨을 예측합니다. 다시 정리하면, 라벨 번호는 다음과 같이 이름지어진 표현에 매핑됩니다.

* `0`: Iris setosa
* `1`: Iris versicolor
* `2`: Iris virginica

In [ ]:
let unlabeledDataset: Tensor<Float> =
    [[5.1, 3.3, 1.7, 0.5],
     [5.9, 3.0, 4.2, 1.5],
     [6.9, 3.1, 5.4, 2.1]]

let unlabeledDatasetPredictions = model(unlabeledDataset)

for i in 0..<unlabeledDatasetPredictions.shape[0] {
    let logits = unlabeledDatasetPredictions[i]
    let classIdx = logits.argmax().scalar!
    print("Example \(i) prediction: \(classNames[Int(classIdx)]) (\(softmax(logits)))")
}